In [1]:
import pandas as pd 
import yfinance as yf 
import plotly.express as px 
import plotly.graph_objs  as go 
from prophet import Prophet 
from stock_object import Stock 
import datetime

## Load one stock data

In [ ]:
def for_prophet(df:pd.DataFrame,date_column='date',y_column='Close'):
    return df.rename(columns={date_column:'ds',y_column:'y'})

In [30]:
TODAY= Stock.nearest_business_day(datetime.date.today())
HORIZON=30
TEST_INTERVAL_LENGTH=200
TRAIN_INTERVAL_LENGTH=1600

TEST_END=Stock.nearest_business_day(TODAY)
TEST_START=Stock.nearest_business_day(TEST_END-datetime.timedelta(days=TEST_INTERVAL_LENGTH))

TRAIN_END=Stock.nearest_business_day(TEST_START-datetime.timedelta(days=1))
TRAIN_START=Stock.nearest_business_day(TRAIN_END-datetime.timedelta(days=TRAIN_INTERVAL_LENGTH))

symbol='GOOG'
stock=Stock(symbol)
train_data=stock.load_data(TRAIN_START,TRAIN_END)
test_data=stock.load_data(TEST_START,TEST_END)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [31]:
df=for_prophet(train_data)
test_df=for_prophet(test_data)

m=Prophet()
m.fit(df)
forecasts=m.predict(test_df)
test_df=test_df.join(forecasts[['yhat_lower','yhat','yhat_upper']])

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [33]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=test_df['ds'],
        y=test_df['y'], 
        mode='lines',
        name='True Closing price'
    )
)

fig.add_trace(
    go.Scatter(
        x=test_df['ds'], 
        y=test_df['yhat'], 
        mode='lines', 
        name='Predicted CLosing price'
    )
)

fig.add_trace(
    go.Scatter(
        x=test_df['ds'], 
        y=test_df['yhat_upper'], 
        fill=None,
        mode='lines', 
        name='CI+', 
        line_color='orange'
    )
)

fig.add_trace(
    go.Scatter(
        x=test_df['ds'], 
        y=test_df['yhat_lower'], 
        fill='tonexty',
        mode='lines', 
        line_color='orange',
        name='CI-'
    )
)
fig.update_layout(template='plotly_dark')
fig.show()